In [2]:
import torch
from torch import nn
from d2l import torch as d2l
from torch.nn import functional as F
from train_epoch.train import train_ch6

In [13]:
class Residual(nn.Module):
    def __init__(self, input_channels, num_channels, use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels, kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels, kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)
    
    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)

In [14]:
blk = Residual(3,3)
X = torch.rand(4, 3, 6, 6)
Y = blk(X)
Y.shape

torch.Size([4, 3, 6, 6])

In [15]:
blk = Residual(3,6, use_1x1conv=True, strides=2)
blk(X).shape

torch.Size([4, 6, 3, 3])

In [16]:
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.BatchNorm2d(64), nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [17]:
def resnet_block(input_channels, num_channels, num_residuals, first_block=False):
    blk = []
    for i in range(num_residuals):
        if i==0 and not first_block:
            blk.append(Residual(input_channels, num_channels, use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

In [18]:
b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
b3 = nn.Sequential(*resnet_block(64, 128, 2))
b4 = nn.Sequential(*resnet_block(128, 256, 2))
b5 = nn.Sequential(*resnet_block(256, 512, 2))

In [19]:
net = nn.Sequential(b1, b2, b3, b4, b5,
                    nn.AdaptiveAvgPool2d((1, 1)),
                    nn.Flatten(), nn.Linear(512, 10))

In [20]:
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 128, 28, 28])
Sequential output shape:	 torch.Size([1, 256, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 512, 1, 1])
Flatten output shape:	 torch.Size([1, 512])
Linear output shape:	 torch.Size([1, 10])


In [21]:
lr, num_epochs, batch_size = 0.05, 10, 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)
train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu(),
          save_path="models/resnet")

new model
training on cuda:0
pretrain test acc: 0.072


Epoch 1/10: 100%|██████████| 60000/60000 [00:09<00:00, 6641.09img/s, loss=0.4825, train_acc=0.8290]


epoch 0, loss 0.482, train acc 0.829, test_acc 0.836


Epoch 2/10: 100%|██████████| 60000/60000 [00:08<00:00, 6812.39img/s, loss=0.2562, train_acc=0.9064]


epoch 1, loss 0.256, train acc 0.906, test_acc 0.834


...

epoch 7, loss 0.040, train acc 0.987, test_acc 0.903


Epoch 9/10: 100%|██████████| 60000/60000 [00:08<00:00, 6898.47img/s, loss=0.0237, train_acc=0.9929]


epoch 8, loss 0.024, train acc 0.993, test_acc 0.919
Saved best model checkpoint to models/resnet\0509123645\best.ckpt with test accuracy 0.919


Epoch 10/10: 100%|██████████| 60000/60000 [00:08<00:00, 6804.13img/s, loss=0.0148, train_acc=0.9960]


epoch 9, loss 0.015, train acc 0.996, test_acc 0.917
loss 0.015, train acc 0.996, test acc 0.917
9023.9 examples/sec on cuda:0
